In [71]:
import importlib
import sys
sys.path.insert(1, 'Osiris Temp\processing\python')
import Analysis_tools as aTools
import rawFileReader
import Reconstructor

<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Peter\AppData\Local\Temp\ipykernel_1056\1135925727.py:3: SyntaxWarning: invalid escape sequence '\p'
  sys.path.insert(1, 'Osiris Temp\processing\python')


In [2]:
#Example Decorators to capture things in the loop so you don't need to redo any calculations later......
def debug_decorator(processedEvents):
    def inner_decorator(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            insertion_list = result
            update = args[0]
            if insertion_list != [0,0,0,0,0]:
                print(f'New alignment, Event chunk {processedEvents}, insertions {insertion_list}, Updates: {update}')
            return result
        return wrapper
    return inner_decorator

In [3]:
importlib.reload(rawFileReader)
interval = 100
file_path = '../../../../Data/Fixed.raw'
fReader = rawFileReader.fileReader(file_path)
order = [[0,1], [1,2], [2,3], [3,4]] #Your order to the TDC alignment
max_process_event_chunk = 100
processedEvents = 0
events = []

#Remember the original function
original_ConstructEventInsertionList = aTools.ConstructEventInsertionList 

while processedEvents < max_process_event_chunk:
    processedEvents += 1
    #Apply your decorator to change the function
    aTools.ConstructEventInsertionList  = debug_decorator(processedEvents)(original_ConstructEventInsertionList)
    event = fReader.get_aligned_events(order=order, interval=interval)
#reset afterwards
aTools.ConstructEventInsertionList  = original_ConstructEventInsertionList


New alignment, Event chunk 1, insertions [0, 0, 0, 1, 3], Updates: [0, 0, -1, -2]
New alignment, Event chunk 3, insertions [0, 4, 8, 8, 8], Updates: [-4, -4, 0, 0]
New alignment, Event chunk 7, insertions [0, 0, 1, 1, 1], Updates: [0, -1, 0, 0]
New alignment, Event chunk 18, insertions [1, 1, 0, 1, 1], Updates: [0, 1, -1, 0]
New alignment, Event chunk 19, insertions [1, 1, 1, 1, 0], Updates: [0, 0, 0, 1]
New alignment, Event chunk 30, insertions [0, 0, 1, 0, 1], Updates: [0, -1, 1, -1]
New alignment, Event chunk 38, insertions [0, 1, 0, 0, 0], Updates: [-1, 1, 0, 0]
New alignment, Event chunk 45, insertions [0, 0, 1, 1, 1], Updates: [0, -1, 0, 0]
New alignment, Event chunk 46, insertions [1, 1, 0, 0, 0], Updates: [0, 1, 0, 0]
New alignment, Event chunk 73, insertions [0, 0, 0, 1, 0], Updates: [0, 0, -1, 1]
New alignment, Event chunk 77, insertions [1, 1, 1, 0, 0], Updates: [0, 0, 1, 0]
New alignment, Event chunk 78, insertions [0, 1, 1, 1, 2], Updates: [-1, 0, 0, -1]
New alignment, Eve

In [72]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(Reconstructor)
interval = 10 # Set your monitoring chunck size
file_path = '../../../../Data/Fixed.raw' # Load in the Raw File
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = Reconstructor.Reconstructor(initial_event_chunk, processedEvents)
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    #Zone of Reconstruction
    if event_chunk:
        reconstructor.update_event(event_chunk, processedEvents)
        reconstructor.populate_hits()
        cluster = reconstructor.make_cluster()
        reconstructor.reconstruct_and_extrapolate(cluster)
    
    # event is a list of proAnubis event object, size {interval}, where it is globally aligned
    # unaligned or realigned chunks are thrown and not outputed
    # can use decorators to capture inner variables like alignment metric. Or set it to global and extract



None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [61]:
reconstructor.possible_reconstructions

[0, 0, 0, 0, 0, 0]

In [50]:
reconstructor.tdcstatus

[True, True, True, True, True]

In [30]:
import matplotlib.pyplot as plt
def plot_efficiency(text = 'Efficiency vs Tolerance error state realign region qual == 0'):
    plt.figure(figsize=(10, 6))
    for RPC in range(6):
        if reconstructor.possible_reconstructions[RPC] == 0:
            efficiency = [0 for x in reconstructor.successful_reconstructions[RPC]]
        else:
            efficiency = [x / reconstructor.possible_reconstructions[RPC] for x in reconstructor.successful_reconstructions[RPC]]
        plt.plot(reconstructor.tol, efficiency, label=f'RPC {RPC}')

    plt.xlabel('Tolerance')
    plt.ylabel('Efficiency')
    plt.title(text)
    plt.legend()
    plt.grid(True)
    plt.show()
    print(reconstructor.possible_reconstructions)

In [31]:
plot_efficiency()

[0, 0, 0, 0, 0, 0]
